In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load the Oxford-IIIT Pet dataset
(ds_train, ds_test), ds_info = tfds.load('oxford_iiit_pet', split=['train', 'test'], as_supervised=True, with_info=True)

# Define a function to resize images
def resize_image(image, label):
    image = tf.image.resize(image, (32, 32))
    return image, label

# Apply resizing to the dataset
ds_train = ds_train.map(resize_image)
ds_test = ds_test.map(resize_image)

# Normalize pixel values to be in the range [0, 1]
ds_train = ds_train.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))
ds_test = ds_test.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))

# Shuffle and batch the datasets
ds_train = ds_train.shuffle(10000).batch(64)
ds_test = ds_test.batch(64)

# Convert labels to categorical format
def categorical_labels(images, labels):
    return images, tf.one_hot(labels, depth=37)  # 37 classes in Oxford-IIIT Pet dataset

ds_train = ds_train.map(categorical_labels)
ds_test = ds_test.map(categorical_labels)

# Verify the shapes
for images, labels in ds_train.take(1):
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)

for images, labels in ds_test.take(1):
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\sadia_tisha1\tensorflow_datasets\oxford_iiit_pet\3.2.0.incompleteNWH99X\oxford_iiit_pet-tra…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\sadia_tisha1\tensorflow_datasets\oxford_iiit_pet\3.2.0.incompleteNWH99X\oxford_iiit_pet-tes…

Dataset oxford_iiit_pet downloaded and prepared to C:\Users\sadia_tisha1\tensorflow_datasets\oxford_iiit_pet\3.2.0. Subsequent calls will reuse this data.
Images shape: (64, 32, 32, 3)
Labels shape: (64, 37)
Images shape: (64, 32, 32, 3)
Labels shape: (64, 37)


In [2]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical


# Create the ResNet model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
    
# Add the top layers for classification
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(37, activation='softmax'))  # Change the number of units and activation
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(ds_train, batch_size=64, epochs=1)

58/58 [==============================] - 120s 2s/step - loss: 4.2437 - accuracy: 0.0565


In [4]:
model.fit(ds_train, batch_size=64, epochs=50)

Epoch 1/50
58/58 [==============================] - 115s 2s/step - loss: 3.5412 - accuracy: 0.1196
Epoch 2/50
58/58 [==============================] - 219s 4s/step - loss: 3.3026 - accuracy: 0.1560
Epoch 3/50
58/58 [==============================] - 293s 5s/step - loss: 2.9172 - accuracy: 0.2484
Epoch 4/50
58/58 [==============================] - 293s 5s/step - loss: 2.7654 - accuracy: 0.3057
Epoch 5/50
58/58 [==============================] - 290s 5s/step - loss: 2.8470 - accuracy: 0.2995
Epoch 6/50
58/58 [==============================] - 192s 3s/step - loss: 2.8860 - accuracy: 0.2897
Epoch 7/50
58/58 [==============================] - 303s 5s/step - loss: 3.2725 - accuracy: 0.2435
Epoch 8/50
58/58 [==============================] - 306s 5s/step - loss: 2.8541 - accuracy: 0.3054
Epoch 9/50
58/58 [==============================] - 294s 5s/step - loss: 2.8222 - accuracy: 0.2755
Epoch 10/50
58/58 [==============================] - 295s 5s/step - loss: 2.3775 - accuracy: 0.3984
Epoch 11/

In [5]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Make predictions on the test dataset
predictions = model.predict(ds_test)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Manually generate ground truth labels for evaluation
true_labels = []
for _, label in ds_test:
    true_labels.extend(label.numpy())

true_labels = np.argmax(true_labels, axis=1)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Assuming you have test_loss and test_accuracy available
# If not, you'll need to calculate them based on your model and test dataset
print(f"Test Accuracy: {accuracy}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")


Test Accuracy: 0.17116380485145816
Accuracy: 0.17116380485145816
F1 Score: 0.1659174869942088
